In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyomo.environ import *

In [2]:
lego_info = pd.read_csv("LEGO_set.csv")

In [3]:
cust_info = pd.read_excel(open('customer.xlsx','rb'), sheet_name='customer')

In [4]:
inventory_info = pd.read_excel(open('customer.xlsx','rb'), sheet_name='inventory')
inventory_list = inventory_info.loc[:, "inventory"].values.tolist()
#len(inventory_list)

In [5]:
quantity_list = cust_info.loc[:, "quantity"].values.tolist()

In [6]:
price_limit = cust_info.loc[:, "price limit"].values.tolist()
#price_limit

In [7]:
lego_price = lego_info.loc[:, "list_price"].values.tolist()

In [8]:
play_rating = lego_info.loc[:, "play_star_rating"].values.tolist()

In [9]:
star_rating = lego_info.loc[:, "star_rating"].values.tolist()

In [10]:
customer_list = cust_info.loc[:, "item 1":].values.tolist()
#customer_list

In [11]:
model = ConcreteModel()
model.x = Var(range(len(lego_info)), range(len(customer_list)), domain = Binary)


In [12]:
#inventory constraint
model.inventory = ConstraintList()
for i in range(len(lego_info)):
    model.inventory.add(expr=sum(model.x[i,j] for j in range(len(customer_list))) <= inventory_list[i])

#customer will not get item that they already have    
model.newsample = ConstraintList()
for j in range(len(customer_list)):
    for i in range(len(lego_info)):
        model.newsample.add(expr=(model.x[i,j] + customer_list[j][i]) <= 1)

#the quantity of lego set should be the same as customer wanted
model.itemcount = ConstraintList()
for j in range(len(customer_list)):
    model.itemcount.add(expr = sum(model.x[i,j] for i in range(len(lego_info))) == quantity_list[j])

#the contraint of price
model.price = ConstraintList()
for j in range(len(customer_list)):
    model.price.add(expr = sum(model.x[i,j]*lego_price[i] for i in range(len(lego_info))) <= price_limit[j])

model.objective = Objective(expr = sum(model.x[i,j]*((play_rating[i]+star_rating[i])/2) for i in range(len(lego_info)) for j in range(len(customer_list))), sense= maximize)
opt = SolverFactory('glpk')
results = opt.solve(model, tee=True)


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/_7/44t0xrr57mx_nrg2qskcmmt40000gn/T/tmpmj31_ygb.glpk.raw
 --wglp /var/folders/_7/44t0xrr57mx_nrg2qskcmmt40000gn/T/tmpz4qbaoq0.glpk.glp
 --cpxlp /var/folders/_7/44t0xrr57mx_nrg2qskcmmt40000gn/T/tmpi_sii31e.pyomo.lp
Reading problem data from '/var/folders/_7/44t0xrr57mx_nrg2qskcmmt40000gn/T/tmpi_sii31e.pyomo.lp'...
/var/folders/_7/44t0xrr57mx_nrg2qskcmmt40000gn/T/tmpi_sii31e.pyomo.lp:902713: warning: lower bound of variable 'x1' redefined
/var/folders/_7/44t0xrr57mx_nrg2qskcmmt40000gn/T/tmpi_sii31e.pyomo.lp:902713: warning: upper bound of variable 'x1' redefined
100901 rows, 100001 columns, 400001 non-zeros
100000 integer variables, all of which are binary
1002713 lines were read
Writing problem data to '/var/folders/_7/44t0xrr57mx_nrg2qskcmmt40000gn/T/tmpz4qbaoq0.glpk.glp'...
801808 lines were written
GLPK Integer Optimizer, v4.65
100901 rows, 100001 columns, 400001 non-zeros
100000 integ

In [13]:
model.solutions.store_to(results)
print(results.Solution.Objective["objective"]["Value"])

5350.0


In [14]:
prod_id = lego_info.loc[:, "prod_id"].values.tolist()
prod_name = lego_info.loc[:, "set_name"].values.tolist()
customer_id = cust_info.loc[:, "customer"].values.tolist()
rating_id = []
for i in range(len(customer_id)):
    rating_id.append(str(customer_id[i]) + "rating")
result_dict = {}
each_list = []
each_avg_rating = 0
total_rating = 0
for j in range(len(customer_list)):
    for i in range(len(lego_info)):
        if model.x[i,j]() == 1:
            each_list.append(str(int(prod_id[i])) + ":" + str(prod_name[i]))
            each_avg_rating += (play_rating[i] + star_rating[i])/2
    result_dict[customer_id[j]] = each_list
    result_dict[rating_id[j]] = each_avg_rating/len(each_list)
    each_list = []
    each_avg_rating = 0

In [15]:
result_dict

{1: ['31059:Sunset Street Bike',
  '41323:Snow Resort Chalet',
  "41316:Andrea's Speedboat Transporter",
  '42072:WHACK!',
  '45502:EV3 Large Servo Motor',
  '45506:EV3 Color Sensor',
  '10857:Piston Cup Race'],
 '1rating': 5.0,
 2: ['10592:Fire Truck',
  "41151:Mulan's Training Day",
  "41316:Andrea's Speedboat Transporter",
  '42072:WHACK!',
  '45502:EV3 Large Servo Motor',
  '45506:EV3 Color Sensor',
  '10857:Piston Cup Race',
  '21143:The Nether Portal'],
 '2rating': 5.0,
 3: ['40171:LEGO® Friends Buildable Hedgehog Storage',
  '31071:Drone Explorer',
  '75887:Porsche 919 Hybrid',
  "10748:Emma's Pet Party"],
 '3rating': 5.0,
 4: ['31059:Sunset Street Bike',
  "41316:Andrea's Speedboat Transporter",
  '42072:WHACK!',
  '45506:EV3 Color Sensor',
  '10857:Piston Cup Race',
  '10847:Number Train',
  '10847:Number Train',
  '60180:Monster Truck',
  '70637:Cole - Spinjitzu Master',
  '76096:Superman™ & Krypto™ Team-Up'],
 '4rating': 5.0,
 5: ["41316:Andrea's Speedboat Transporter",
  '4